In [1]:
import os

In [2]:
%pwd

'c:\\Users\\user\\Desktop\\Insurance E2E ML\\Insurance-Premium-End-to-End-\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'c:\\Users\\user\\Desktop\\Insurance E2E ML\\Insurance-Premium-End-to-End-'

In [11]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    train_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str

In [12]:
from src.inproject.constants import *
from src.inproject.utils.common import read_yaml, create_directories

In [13]:
# for src configuration
class ConfigurationManager:
    def __init__(self, 
                 config_pathway=CONFIG_FILE_PATH, 
                 param_pathway=PARAM_FILE_PATH, 
                 schema_pathway=SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_pathway)
        self.param = read_yaml(param_pathway)
        self.schema = read_yaml(schema_pathway)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
            config = self.config.model_evaluation
            params = self.param.LGBMRegressor
            schema = self.schema.TARGET_COLUMN

            create_directories([config.root_dir])

            data_evaluation_config = ModelEvaluationConfig(
                root_dir = config.root_dir,
                test_data_path=config.test_data_path,
                train_data_path=config.train_data_path,
                model_path=config.model_path,
                all_params=params,
                metric_file_name=config.metric_file_name,
                target_column=schema.name

            )

            return data_evaluation_config


In [14]:
# import os
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from urllib.parse import urlparse
import numpy as np 
import joblib
from src.inproject.utils.common import save_json


In [15]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config
    
    def eval_metrics(self, actual, pred):
        rmse=np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)

        return rmse, mae, r2
    
    def save_result(self):
    # Load test data and the trained model
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        # Prepare test features and target
        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]

        # Predictions and metrics for the test data
        predicted_test = model.predict(test_x)
        (rmse_test, mae_test, r2_test) = self.eval_metrics(test_y, predicted_test)

        # Load training data
        train_data = pd.read_csv(self.config.train_data_path)
        train_x = train_data.drop([self.config.target_column], axis=1)
        train_y = train_data[[self.config.target_column]]

        # Predictions and metrics for the training data
        predicted_train = model.predict(train_x)
        (rmse_train, mae_train, r2_train) = self.eval_metrics(train_y, predicted_train)

        # Save metrics for both training and testing
        scores = {
            'train': {'rmse': rmse_train, 'mae': mae_train, 'r2': r2_train},
            'test': {'rmse': rmse_test, 'mae': mae_test, 'r2': r2_test}
        }

        # Save the metrics to a JSON file
        save_json(path=Path(self.config.metric_file_name), data=scores)


In [16]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.save_result()

except Exception as e:
    raise e

[2025-01-03 00:10:19,343: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-01-03 00:10:19,348: INFO: common: yaml file: params.yaml loaded successfully]


[2025-01-03 00:10:19,357: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-01-03 00:10:19,357: INFO: common: created directory at: artifacts]
[2025-01-03 00:10:19,365: INFO: common: created directory at: artifacts/model_evaluation]
[2025-01-03 00:10:31,026: INFO: common: json file saved at: artifacts\model_evaluation\metric.json]
